In [1]:
%%bash
pip install xarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 972.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=436ec8247805af7ff64f43552021e1ea329897730ee583740d7526441fea594c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
#from sparkxarray.reader import ncread
from pyspark.sql import SparkSession
import xarray as xr
import os
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.types import * 
from pyspark.sql import *


#need to add config variables to create multi nodes
spark = SparkSession.builder.appName('spark-rdd').getOrCreate()


In [6]:
# need to take input from entire folder, rather then hardcoded filename
# CREATE a loop that calls function to iterate through each file in folder(sort folder by name)
#  Pseudo: for n files in folder, grab both file types, transform, append results to merged csv


#TODO add queries for individual input variables with averages results, group by lat lon; append to temp table
# take results from temp table and append to master file csv
# create local variable To store date from file, append to last column of temp table


A3paths = os.listdir('/content/input/A3/')
I3paths = os.listdir('/content/input/I3/')
A3paths.sort()
I3paths.sort() 

for index, (value1, value2) in enumerate(zip(A3paths, I3paths)):
  #append to Log File?
  print(value1)
  print(value2)

  #for prod, folders will need to be sorted bv filename, possibly add a conditonal to check that dates from names match for I3 = A3
  dsA3 = xr.open_dataset("input/A3/" + value1)#, combine_attrs= 'identical'
  dsI3 = xr.open_dataset("input/I3/" +value2 )
  dtA3 = str(dsA3.get_index(key='time')[0])[:10]
  dtI3 = str(dsI3.get_index(key='time')[0])[:10]


  if dtI3 == dtA3:
    dsA3 = dsA3.where(dsA3.lev >= 1.0, drop=True).where(dsA3.lev < 2.0 , drop=True).to_dataframe()
    dfI3 = dsI3.where(dsI3.lev >= 1.0, drop=True).where(dsI3.lev < 2.0 , drop=True).to_dataframe()

    #convert df to df 1d array, index and values
    stkDsA3 = dsA3.stack()
    stkDsI3 = dfI3.stack()

    # Split indexes and values to 2 Spark df
    dfA3indx = spark.createDataFrame(stkDsA3.index)
    dfA3val = spark.createDataFrame(stkDsA3.values)

    dfI3indx = spark.createDataFrame(stkDsI3.index)
    dfI3val = spark.createDataFrame(stkDsI3.values)

    # add row number column to each data frame for I3 file
    w = Window().orderBy(lit('A'))
    dfA3indx = dfA3indx.withColumn("row_numI", row_number().over(w))
    dfA3val = dfA3val.withColumn("row_numV", row_number().over(w))

    dfI3indx = dfI3indx.withColumn("row_numI", row_number().over(w))
    dfI3val = dfI3val.withColumn("row_numV", row_number().over(w))

    dfA3indx.createOrReplaceTempView("abc")
    dfA3val.createOrReplaceTempView("cde")

    #merge indx and val to single spark data frame based on row number column
    mergA3 = spark.sql("select * from abc d1 join cde d1f on d1.row_numI = d1f.row_numV ").withColumnRenamed('_1','time').withColumnRenamed(
        '_2', 'level').withColumnRenamed('_3', 'lat').withColumnRenamed('_4','long').withColumnRenamed('_5','type')
    mergA3.orderBy("row_numV").show(truncate=False)

    dfI3indx.createOrReplaceTempView("abc")
    dfI3val.createOrReplaceTempView("cde")

    mergI3 = spark.sql("select * from abc d1 join cde d1f on d1.row_numI = d1f.row_numV ").withColumnRenamed('_1','time').withColumnRenamed(
        '_2', 'lat').withColumnRenamed('_3', 'long').withColumnRenamed('_4','level').withColumnRenamed('_5','type')
    mergI3.orderBy("row_numV").show(truncate=False)

    #average results
    tmp = mergA3.select("lat","long","type","value").where("type == 'RH'")
    temp1 = tmp.groupBy("lat","long").avg('value')
    rHDf = temp1.orderBy('lat').orderBy('long').withColumnRenamed("avg(value)", "value")

    tmp = mergA3.select("lat","long","type","value").where("type == 'U'")
    temp1 = tmp.groupBy("lat","long").avg('value')
    uDf = temp1.orderBy('lat').orderBy('long').withColumnRenamed("avg(value)", "value")

    tmp = mergA3.select("lat","long","type","value").where("type == 'V'")
    temp1 = tmp.groupBy("lat","long").avg('value')
    vDf = temp1.orderBy('lat').orderBy('long').withColumnRenamed("avg(value)", "value")

    tmp = mergI3.select("lat","long","type","value").where("type == 'QV'")
    temp1 = tmp.groupBy("lat","long").avg('value')
    qVDf = temp1.orderBy('lat').orderBy('long').withColumnRenamed("avg(value)", "value")

    tmp = mergI3.select("lat","long","type","value").where("type == 'T'")
    temp1 = tmp.groupBy("lat","long").avg('value')
    tDf = temp1.orderBy('lat').orderBy('long').withColumnRenamed("avg(value)", "value")
    tDf= tDf.withColumn("date", lit(dtA3))


    tDf.show()

    #temp table needs to be lat|lon|RH|U|V|QV|T|date
    #to add weighted mean column, add coordinated overlay grid to df, run calculation of weighted mean from any of tmp dataframes reltive to overlay grid df
    # take weighted mean dataframe and createorreplacetempview
    # add temp view to master query and add the tempview table as final column to be returned to finalDf
    rHDf.createOrReplaceTempView('RH')
    uDf.createOrReplaceTempView('U')
    vDf.createOrReplaceTempView('V')
    qVDf.createOrReplaceTempView('QV')
    tDf.createOrReplaceTempView('T')

    finalDf= spark.sql("select rh.lat as lat, rh.long as long, rh.value as R, u.value as U, " +
              " v.value as V, qv.value as QV, t.value as T, t.date as date "
              " from RH rh join U u on rh.lat = u.lat and rh.long = u.long " +
              " join V v on rh.lat = v.lat and rh.long = v.long " + 
              " join QV qv on rh.lat = qv.lat and rh.long = qv.long " + 
              " join T t on rh.lat = t.lat and rh.long = t.long")

    finalDf.show()
    finalDf.write.format("csv").mode('append').save("/content/output")

  

MERRA2.19880101.A3dyn.2x25.nc4
MERRA2.19880101.I3.2x25.nc4
+----+-----+-----+------+------+--------+------------+--------+
|time|level|lat  |long  |type  |row_numI|value       |row_numV|
+----+-----+-----+------+------+--------+------------+--------+
|{}  |1.0  |-90.0|-180.0|DTRAIN|1       |0.0         |1       |
|{}  |1.0  |-90.0|-180.0|OMEGA |2       |-0.029221918|2       |
|{}  |1.0  |-90.0|-180.0|RH    |3       |0.83456624  |3       |
|{}  |1.0  |-90.0|-180.0|U     |4       |-0.88576674 |4       |
|{}  |1.0  |-90.0|-180.0|V     |5       |0.1761617   |5       |
|{}  |1.0  |-90.0|-177.5|DTRAIN|6       |0.0         |6       |
|{}  |1.0  |-90.0|-177.5|OMEGA |7       |-0.029221918|7       |
|{}  |1.0  |-90.0|-177.5|RH    |8       |0.83456624  |8       |
|{}  |1.0  |-90.0|-177.5|U     |9       |-0.88576674 |9       |
|{}  |1.0  |-90.0|-177.5|V     |10      |0.1761617   |10      |
|{}  |1.0  |-90.0|-175.0|DTRAIN|11      |0.0         |11      |
|{}  |1.0  |-90.0|-175.0|OMEGA |12      |-0.0

OSError: ignored

#TESTING